# Overview

This notebook contains my solutions for **<a href="https://adventofcode.com/2024" target="_blank">Advent of Code 2024</a>**.

A few notes...
- The source for this notebook source lives in my GitHub repo, <a href="https://github.com/derailed-dash/Advent-of-Code/blob/master/src/AoC_2024/Dazbo's_Advent_of_Code_2024.ipynb" target="_blank">here</a>.
- You can run this Notebook wherever you like. For example, you could...
  - Run it locally, in your own Jupyter environment.
  - Run it in a cloud-based Jupyter environment, with no setup required on your part!  For example, with **Google Colab**: <br><br><a href="https://colab.research.google.com/github/derailed-dash/Advent-of-Code/blob/master/src/AoC_2024/Dazbo's_Advent_of_Code_2024.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Google Colab"/></a><br>
  - For more ways to run Jupyter Notebooks, check out [my guide](https://medium.com/python-in-plain-english/five-ways-to-run-jupyter-labs-and-notebooks-23209f71e5c0).
- **To run the notebook, execute the cells in the [Setup](#Setup) section, as described below. Then you can run the code for any given day.**
- Be mindful that the first time you run this notebook, you will need to **obtain your AoC session key** and store it, if you have not done so already. This allows the notebook to automatically retrieve your input data. (See the guidance in the **[Get Access to Your AoC Data](#Get-Access-to-Your-AoC-Data)** section for details.)
- Use the navigation menu on the left to jump to any particular day.
- All of my AoC solutions are documented in my <a href="https://aoc.just2good.co.uk/" target="_blank">AoC Python Walkthrough site</a>.

# Setup

You need to run all cells in this section, before running any particular day solution.

## Packages and Imports

Here we use `pip` to install the packages used by my solutions in this event. After installing the packages, you may need to restart your Jupyter kernel in order for the packages to be detected and remove any linting errors.

In [1]:
%pip install --upgrade --no-cache-dir \
    jupyterlab-lsp ipykernel \
    matplotlib pandas networkx  sympy \
    dazbo-commons \
    python-dotenv tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 37.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.0
    Uninstalling tqdm-4.67.0:
      Successfully uninstalled tqdm-4.67.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.9.2
    Uninstalling matplotlib-3.9.2:
      Successfully uninstalled matplotlib-3.9.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import annotations
import ast
import copy
import heapq
import logging
import math
import operator
import os
import platform
import re
from collections import Counter, deque, defaultdict
from dataclasses import asdict, dataclass, field
from enum import Enum, auto
from functools import cache, reduce
from itertools import combinations, count, cycle, permutations
from getpass import getpass
from pathlib import Path

# Third-party imports
import dazbo_commons as dc  # my own utility library, which includes things like coloured logging
import requests
import sympy
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from IPython.core.display import Markdown
from IPython.display import display
from tqdm.notebook import tqdm

## Logging and Output

Set up a new logger that uses `ColouredFormatter`, such that we have coloured logging.  The log colour depends on the logging level.

In [3]:
# Setup logger
YEAR = 2024
APP_NAME = "aoc" + str(YEAR)
logger = dc.retrieve_console_logger(APP_NAME)
logger.setLevel(logging.DEBUG)
logger.info("Logger initialised.")
logger.debug("Debugging enabled")

16:56:13.686:aoc2024 - INF: Logger initialised.
16:56:13.687:aoc2024 - DBG: Debugging enabled


## Install Packages

- [ffmpeg](https://ffmpeg.org/): in order to render video output, i.e. for visualisations.
- graphviz: for visualising graphs

In [4]:
import subprocess

def run_command(command):
    """Run a shell command and print its output in real-time."""
    process = subprocess.Popen(
        command, 
        shell=True, 
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE
    )
    
    # Read and print the output line by line
    if process.stdout is not None:
        for line in iter(process.stdout.readline, b''):
            logger.info(line.decode().strip())
        process.stdout.close()
        
    process.wait()
    
def install_software(appname: str):
    os_name = platform.system()
    logger.info(f"Installing {appname} on {os_name}...")
    
    # Mapping operating systems to their respective installation commands
    command_map = {
        "Windows": f"winget install {appname} --silent --no-upgrade",
        "Linux": f"apt -qq -y install {appname}",
        "Darwin": f"brew install {appname}"
    }
    command = command_map.get(os_name)
    if command:
        run_command(command)
    else:
        logger.error(f"Unsupported operating system: {os_name}")

def check_installed(app_exec: str) -> bool:    
    appname, *arg = app_exec.split()
    arg = " ".join(arg)
    logger.debug(f"Checking if {appname} is installed")
    
    try:
        output = subprocess.check_output([appname, arg], stderr=subprocess.STDOUT)
        logger.debug(f"{appname} version: {output.decode().strip()}")
        logger.debug(f"{appname} is already installed.")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        logger.debug(f"{appname} is not installed or absent from path.")
        
    return False

apps = [ ("ffmpeg", "ffmpeg -version"),
         ("graphviz", "dot --version") ]
          
for app_install, app_exec in apps:
    if not check_installed(app_exec):
        install_software(app_install)


16:56:13.696:aoc2024 - DBG: Checking if ffmpeg is installed
16:56:13.904:aoc2024 - DBG: ffmpeg version: ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/aarch64-linux-gnu --incdir=/usr/include/aarch64-linux-gnu --arch=arm64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --en

## Load Helpers and Useful Classes

Now we load a bunch of helper functions and classes.

### Locations

Where any input and output files get stored.

<img src="https://aoc.just2good.co.uk/assets/images/notebook-content-screenshot.png" width="320" />


### Retrieve the Input Data

This works by using your unique session cookie to retrieve your input data. E.g. from a URL like:

`https://adventofcode.com/2024/day/1/input`

In [5]:
##################################################################
# Retrieving input data
##################################################################

def write_puzzle_input_file(year: int, day, locations: dc.Locations):
    """ Use session key to obtain user's unique data for this year and day.
    Only retrieve if the input file does not already exist.
    Return True if successful.
    Requires env: AOC_SESSION_COOKIE, which can be set from the .env.
    """
    if os.path.exists(locations.input_file):
        logger.debug("%s already exists", os.path.basename(locations.input_file))
        return os.path.basename(locations.input_file)

    session_cookie = os.getenv('AOC_SESSION_COOKIE')
    if not session_cookie:
        raise ValueError("Could not retrieve session cookie.")

    logger.info('Session cookie retrieved: %s...%s', session_cookie[0:6], session_cookie[-6:])

    # Create input folder, if it doesn't exist
    if not locations.input_dir.exists():
        locations.input_dir.mkdir(parents=True, exist_ok=True)

    url = f"https://adventofcode.com/{year}/day/{day}/input"
    
    # Don't think we need to set a user-agent
    # headers = {
    #     "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'
    # }
    cookies = { 
        "session": session_cookie
    }
    response = requests.get(url, cookies=cookies, timeout=5)

    data = ""
    if response.status_code == 200:
        data = response.text

        with open(locations.input_file, 'w') as file:
            logger.debug("Writing input file %s", os.path.basename(locations.input_file))
            file.write(data)
            return data
    else:
        raise ValueError(f"Unable to retrieve input data.\n" +
                         f"HTTP response: {response.status_code}\n" +
                         f"{response.reason}: {response.content.decode('utf-8').strip()}")


### Testing

A really simple function for testing that our solution produces the expected test output. If the `validate()` call fails, then execution will stop.

In [6]:
def validate(test, answer):
    """
    Args:
        test: the answer given by our solution
        answer: the expected answer, e.g. from instructions
    """
    if test != answer:
        raise AssertionError(f"{test} != {answer}")

### Useful Helper Classes

In [7]:
#################################################################
# POINTS, VECTORS AND GRIDS
#################################################################

@dataclass(frozen=True)
class Point:
    """ Class for storing a point x,y coordinate """
    x: int
    y: int

    def __add__(self, other: Point):
        return Point(self.x + other.x, self.y + other.y)

    def __mul__(self, other: Point):
        """ (x, y) * (a, b) = (xa, yb) """
        return Point(self.x * other.x, self.y * other.y)

    def __sub__(self, other: Point):
        return self + Point(-other.x, -other.y)

    def __lt__(self, other):
        # Arbitrary comparison logic
        return (self.x, self.y) < (other.x, other.y)
    
    def yield_neighbours(self, include_diagonals=True, include_self=False):
        """ Generator to yield neighbouring Points """

        deltas: list
        if not include_diagonals:
            deltas = [vector.value for vector in Vectors if abs(vector.value[0]) != abs(vector.value[1])]
        else:
            deltas = [vector.value for vector in Vectors]

        if include_self:
            deltas.append((0, 0))

        for delta in deltas:
            yield Point(self.x + delta[0], self.y + delta[1])

    def neighbours(self, include_diagonals=True, include_self=False) -> list[Point]:
        """ Return all the neighbours, with specified constraints.
        It wraps the generator with a list. """
        return list(self.yield_neighbours(include_diagonals, include_self))

    def get_specific_neighbours(self, directions: list[Vectors]) -> list[Point]:
        """ Get neighbours, given a specific list of allowed locations """
        return [(self + Point(*vector.value)) for vector in list(directions)]

    @staticmethod
    def manhattan_distance(a_point: Point) -> int:
        """ Return the Manhattan distance value of this vector """
        return sum(abs(coord) for coord in asdict(a_point).values())

    def manhattan_distance_from(self, other: Point) -> int:
        """ Manhattan distance between this Vector and another Vector """
        diff = self-other
        return Point.manhattan_distance(diff)

    def __repr__(self):
        return f"P({self.x},{self.y})"

class Vectors(Enum):
    """ Enumeration of 8 directions.
    Note: y axis increments in the North direction, i.e. N = (0, 1) """
    N = (0, 1)
    NE = (1, 1)
    E = (1, 0)
    SE = (1, -1)
    S = (0, -1)
    SW = (-1, -1)
    W = (-1, 0)
    NW = (-1, 1)

    @property
    def y_inverted(self):
        """ Return vector, but with y-axis inverted. I.e. N = (0, -1) """
        x, y = self.value
        return (x, -y)

class VectorDicts():
    """ Contains constants for Vectors """
    ARROWS = {
        '^': Vectors.N.value,
        '>': Vectors.E.value,
        'v': Vectors.S.value,
        '<': Vectors.W.value
    }

    DIRS = {
        'U': Vectors.N.value,
        'R': Vectors.E.value,
        'D': Vectors.S.value,
        'L': Vectors.W.value
    }

    NINE_BOX: dict[str, tuple[int, int]] = {
        # x, y vector for adjacent locations
        'tr': (1, 1),
        'mr': (1, 0),
        'br': (1, -1),
        'bm': (0, -1),
        'bl': (-1, -1),
        'ml': (-1, 0),
        'tl': (-1, 1),
        'tm': (0, 1)
    }

class Grid():
    """ 2D grid of point values. """
    def __init__(self, grid_array: list) -> None:
        self._array = grid_array
        self._width = len(self._array[0])
        self._height = len(self._array)

    def value_at_point(self, point: Point) -> int:
        """ The value at this point """
        return self._array[point.y][point.x]

    def set_value_at_point(self, point: Point, value: int):
        self._array[point.y][point.x] = value

    def valid_location(self, point: Point) -> bool:
        """ Check if a location is within the grid """
        if (0 <= point.x < self._width and  0 <= point.y < self._height):
            return True

        return False

    @property
    def width(self):
        """ Array width (cols) """
        return self._width

    @property
    def height(self):
        """ Array height (rows) """
        return self._height

    def all_points(self) -> list[Point]:
        points = [Point(x, y) for x in range(self.width) for y in range(self.height)]
        return points

    def rows_as_str(self):
        """ Return the grid """
        return ["".join(str(char) for char in row) for row in self._array]

    def cols_as_str(self):
        """ Render columns as str. Returns: list of str """
        cols_list = list(zip(*self._array))
        return ["".join(str(char) for char in col) for col in cols_list]

    def __repr__(self) -> str:
        return f"Grid(size={self.width}*{self.height})"

    def __str__(self) -> str:
        return "\n".join("".join(map(str, row)) for row in self._array)

### Useful Helper Functions

In [8]:
#################################################################
# CONSOLE STUFF
#################################################################

def cls():
    """ Clear console """
    os.system('cls' if os.name=='nt' else 'clear')

#################################################################
# USEFUL FUNCTIONS
#################################################################

def binary_search(target, low:int, high:int, func, *func_args, reverse_search=False):
    """ Generic binary search function that takes a target to find,
    low and high values to start with, and a function to run, plus its args.
    Implicitly returns None if the search is exceeded. """

    res = None  # just set it to something that isn't the target
    candidate = 0  # initialise; we'll set it to the mid point in a second

    while low < high:  # search exceeded
        candidate = int((low+high) // 2)  # pick mid-point of our low and high
        res = func(candidate, *func_args) # run our function, whatever it is
        logger.debug("%d -> %d", candidate, res)
        if res == target:
            return candidate  # solution found

        comp = operator.lt if not reverse_search else operator.gt
        if comp(res, target):
            low = candidate
        else:
            high = candidate

def merge_intervals(intervals: list[list]) -> list[list]:
    """ Takes intervals in the form [[a, b][c, d][d, e]...]
    Intervals can overlap.  Compresses to minimum number of non-overlapping intervals. """
    intervals.sort()
    stack = []
    stack.append(intervals[0])

    for interval in intervals[1:]:
        # Check for overlapping interval
        if stack[-1][0] <= interval[0] <= stack[-1][-1]:
            stack[-1][-1] = max(stack[-1][-1], interval[-1])
        else:
            stack.append(interval)

    return stack

@cache
def get_factors(num: int) -> set[int]:
    """ Gets the factors for a given number. Returns a set[int] of factors.
        # E.g. when num=8, factors will be 1, 2, 4, 8 """
    factors = set()

    # Iterate from 1 to sqrt of 8,
    # since a larger factor of num must be a multiple of a smaller factor already checked
    for i in range(1, int(num**0.5) + 1):  # e.g. with num=8, this is range(1, 3)
        if num % i == 0: # if it is a factor, then dividing num by it will yield no remainder
            factors.add(i)  # e.g. 1, 2
            factors.add(num//i)  # i.e. 8//1 = 8, 8//2 = 4

    return factors

def to_base_n(number: int, base: int):
    """ Convert any integer number into a base-n string representation of that number.
    E.g. to_base_n(38, 5) = 123

    Args:
        number (int): The number to convert
        base (int): The base to apply

    Returns:
        [str]: The string representation of the number
    """
    ret_str = ""
    curr_num = number
    while curr_num:
        ret_str = str(curr_num % base) + ret_str
        curr_num //= base

    return ret_str if number > 0 else "0"


# Env Clear

Only run the next cell if you want to manually clear your session key.

In [ ]:
del os.environ['AOC_SESSION_COOKIE']

# Days

Here you'll find a template to build a solution for a given day, and then the solutions for all days in this event.

To copy the template day, select all the cells in the `Day n` template, add a new cell at the end, and then paste the cells there.

---
## Day 1: Historian Hysteria

In [ ]:
DAY = "1" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 1 Part 1

We're told: historically significant locations are listed by a unique number called the location ID. The Historians have split into two groups, and each group has created their own list of locations.

Our puzzle input is the two lists; i.e. presented as two columns of numbers. We need to reconcile the differences. E.g.

```text
3   4
4   3
2   5
1   3
3   9
3   3
```

Goal:

- Pair the smallest number in the two lists, and measure the difference.
- Then the second smallest number from each list, and so on.
- Return the sum of differences.

**What is the total distance between your lists?**

#### Solution Approach

1. Build two lists by splitting each line into two numbers. We can easily do this by [reading each line](https://aoc.just2good.co.uk/python/reading_files), and calling the `split()` function, which automatically splits on the whitespace between the numbers. For each row, we now end up with two numbers, but returned as string values. So we use [map()](https://aoc.just2good.co.uk/python/map-filter-reduce) to turn the strings into integer types. Now we have a pair of integer numbers for each line, which we wrap in a tuple and return in a [list comprehension](https://aoc.just2good.co.uk/python/comprehensions). I.e. one list containing tuples, which are each a pair of integers. Finally, we use the [zip()](https://aoc.just2good.co.uk/python/zip) function to turn the list of tuples pairs into a pair of lists: one list for left column, and one list for the right column.
1. Sort the two lists, such that the smallest number in each column comes first, and so on.
1. Find the absolute difference between each pair of numbers, and sum these differences.

In [ ]:
@cache
def process_data(data) -> tuple[list[int], list[int]]:
    items = [tuple(map(int, line.split())) for line in data] # returns list of tuples, e.g. [(3, 5), ...]
    list1, list2 = zip(*items) # list of all lefts, plus list of all rights
    return sorted(list1), sorted(list2)

def solve_part1(data) -> int:
    list1, list2 = process_data(tuple(data))
    
    diff = 0
    for item1, item2 in zip(list1, list2):
        diff += abs(item2 - item1)
        
    return diff

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""\
3   4
4   3
2   5
1   3
3   9
3   3
""")
sample_answers = [11]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 1 Part 2

Figure out exactly how often each number from the left list appears in the right list. Calculate a total similarity score by adding up each number in the left list after multiplying it by the number of times that number appears in the right list.

**What is their similarity score?**

#### Solution Approach

1. For each item in list1, count how many times it appears in list2.
1. Add the product of the item and the count to our score.


In [ ]:
def solve_part2(data):
    list1, list2 = process_data(tuple(data))
    
    score = 0
    for item in list1:
        item_count = list2.count(item)
        score += item * item_count
        
    return score

In [ ]:
%%time

sample_answers = [31]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

### Alternative Solutions

We could also solve using [Numpy](https://aoc.just2good.co.uk/python/numpy). This will be more efficient for large arrays.

In [ ]:
def solve(data) -> int:
    sorted_lists = process_data(tuple(data))
    
    # Create two Numpy arrays
    list1 = np.array(sorted_lists[0])
    list2 = np.array(sorted_lists[1])
    
    diff = np.sum(np.abs(list2-list1)) # The sum of item diffs
    logger.info(f"{diff=}")
    
    similarity_score = 0
    for item in list1:
        item_count = (list2 == item).sum() # How many times the item appears in list2
        similarity_score += item * item_count
        
    logger.info(f"{similarity_score=}")
    
solve(input_data)

---
## Day 2: Red-Nosed Reports

In [ ]:
DAY = "2" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 2 Part 1

Our data is a set of reports. Each line is one report. Each report contains a list of numbers called levels. Like this:

```text
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9
```

Reports are safe if:

- The levels are either all increasing or all decreasing
- And any two adjacent levels must differ by at least 1 and at most 3.

**How many reports are safe?**

#### Solution Approach

- Split each record into a list of int values, called `levels`.
- Create `is_safe()` function which:
  - Creates a list of difference values - called `diffs` - by subtracting `levels` with offset 1 from `levels`.
  - Then, use the `all` iterator operator function to check if our condition is true for every diff in `diffs`.
  - The condition is: that every diff is positive and `<=MAX_DIFF` or every diff is negative and `>=MAX_DIFF`.

In [ ]:
def is_safe(levels: list[int], max_diff: int) -> bool:
    """ Determine if a record - i.e. a list of levels - is safe.
    Do this by determining the differences between each level in the record.
    Check if the differences are all positive and within the max, 
    or all negative and within the max. """
    
    # Create pairs of levels from successive levels
    # For each pair, substract second from first to get the difference
    diffs = [first-second for first, second in zip(levels, levels[1:])]
    
    return (all(0 < diff <= max_diff for diff in diffs) or 
            all(-max_diff <= diff < 0 for diff in diffs))

def solve_part1(data):
    safe_count = 0
    for report in data:
        levels = list(map(int, report.split()))
        if is_safe(levels, 3):
            safe_count += 1
    
    return safe_count

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""\
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9""")
sample_answers = [2]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 2 Part 2

We now have a _Problem Dampener_, which means we can tolerate a single bad level in any given record.

So we must try removing levels from records, and then testing if the report is safe.

**How many reports are now safe?**

#### Solution Approach

Same as before, but this time we will iterate through the index values of each record, and remove that level from the record. I.e.

- Loop through each index value for the record.
- For the current index value, concatenate the levels up to this index with the levels after this index. Thus, we always end up with a list that is one shorter than the original list. (Note that trimming off the first or last level will never make a safe record _unsafe_.)
- Pass shortened list to our `is_safe()` function.
- If the shortened record is safe, then we can add this record to our counter and move on to the next record.

In [ ]:
def solve_part2(data):
    safe_count = 0
    for report in data:
        levels = list(map(int, report.split()))

        # Slice out one level at a time, and check if the new record is safe
        for idx in range(len(levels)):
            # Take levels up this idx, and concatenate with levels AFTER this idx
            trimmed_levels = levels[:idx] + levels[idx+1:]
            if is_safe(trimmed_levels, 3):
                safe_count += 1
                break
    
    return safe_count

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""\
7 6 4 2 1
1 2 7 8 9
9 7 6 2 1
1 3 2 4 5
8 6 4 4 1
1 3 6 7 9""")
sample_answers = [4]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

---
## Day 3: Mull It Over

In [ ]:
DAY = "3" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 3 Part 1

A shop computer is trying to run a program, but its memory - the puzzle input - is corrupted.

It should be doing instructions like:

- `mul(X,Y)` where `X` and `Y` are 1-3 digit numbers.

Our instructions:

- There are many invalid characters which should be ignored.
- If an invalid character is part of a `mul()` instruction, then the whole instruction does nothing.
- Scan the corrupted memory for uncorrupted mul instructions.

**What do you get if you add up all of the results of the multiplications?**

#### Solution Approach

We can just use [regex](https://aoc.just2good.co.uk/python/regex).

- Use a regex pattern to identify substrints that match the required input format.
- Use the regex `finditer()` to retrieve all non-overlapping matches.
- The `matches.groups()` retrieves the captured groups, i.e. the digits themselves. We identify capture groups by placing each digit in brackets in the pattern string.
- Then, multiply the two digits together, as required.

In [ ]:
def solve_part1(data):
    # match all "mul(x,y)"
    matches = re.finditer(r"mul\((\d{1,3}),(\d{1,3})\)", data)
    
    ans = 0
    for match in matches:
        val_x, val_y = match.groups()
        ans += int(val_x)*int(val_y)
        
    return ans


In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""xmul(2,4)%&mul[3,7]!@^do_not_mul(5,5)+mul(32,64]then(mul(11,8)mul(8,5))""")
sample_answers = [161]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day 3 Part 2

Now we also want to handle intact conditional statements. We need to handle:

- `do()` which enables future `mul` instructions
- `don't()` which disables future `mul` instructions.

We start enabled.

**What is the sum of just the enabled multiplications?**

#### Solution Approach

- Okay, we start `enabled`.
- Whilst `enabled==True`, everything up to the next `don't()` should be parsed for `mul()` instructions.
- When we hit a `don't()`, we set `enabled=False`.
- Now, everything up to the next `do()` should be ignored.
- When we hit a `do()`, we set `enabled=True` again.

I'm shocked... This worked first time with no bugs!!

In [ ]:
def solve_part2(data: str):
    # match all "mul(x,y)"
    matcher = re.compile(r"mul\((\d{1,3}),(\d{1,3})\)")
    
    remaining_program = data
    enabled = True # We start enabled
    ans = 0
    while remaining_program:
        if enabled:
            # split into the part before the next "don't()", and everything afterwards
            this_part, _, remaining_program = remaining_program.partition(r"don't()")
        
            for match in matcher.finditer(this_part):
                val_x, val_y = match.groups()
                ans += int(val_x)*int(val_y)
            
            enabled = False
        else:
            # split into the part before the next "do()", and everything afterwards
            this_part, _, remaining_program = remaining_program.partition(r"do()")
            enabled = True
        
    return ans

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))""")
sample_answers = [48]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

---
## Day 4: Ceres Search

In [ ]:
DAY = "4" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day 4 Part 1

We have a word search! We need to find every instance of the word `XMAS`. The string can be found horizontally, vertically, diagonally, and in forward/reverse directions.

#### Solution Approach

- Create a dictinary to count how many times a given character is part of a matched word. Why did I do this?
  - Firstly, we can use this dictionary to determine if any given character in the grid has been matched in a word. This makes it easy to print a grid that only shows matched characters. Useful for debugging!
  - Secondly, I thought Part 2 might be asking where we have characters that are matched more than once, i.e. where words cross. This dict would have made it super easy for me to determine that. Alas, my guess was wrong!
- Now move through each location in the grid, and stop whenever we find the `X` of `XMAS`.
- Having identified the starting `X`, now iterate through all 8 possible directions.
- For each direction, first determine if the full word can fit in the grid, i.e. that we don't get cut off by a grid edge.
- If the word can fit, we then move one char at a time in the current direction, and test if the characters match the remaining characters in our word. 
- If all characters match, then we've matched the word and we can increment that `matches` counter.  But also, increment the counts for each character in this word.
- Now, render a visualisation of the grid, showing a `.` whenever the count at this location is 0, else show the character at this location.#
- Finally, return the total number of `matches`.


In [ ]:
def solve_part1(data):
    match_word = "XMAS"
    matched_posn_counts = defaultdict(int) # increment whenever this index is in a matched word

    grid_width = len(data[0])
    grid_height = len(data)

    matches = 0

    # Iterate through all positions in the grid
    for row_idx in range(grid_height):
        for col_idx in range(grid_width):
            # Find the X (first char)
            if data[row_idx][col_idx] == match_word[0]:
                # Now we want to expand in all directions
                for direction in Vectors: # Loop through all directions

                    # Check if the full word can be within the grid bounds
                    chars_left = len(match_word)-1
                    if not (0 <= (col_idx + chars_left*direction.value[0]) < grid_width):
                        continue # try next direction
                    if not (0 <= (row_idx + chars_left*direction.value[1]) < grid_height):
                        continue # try next direction

                    found_word = True

                    # Test remaining letters in this direction
                    for steps, char in enumerate(match_word[1:], start=1):
                        new_row_idx = row_idx + steps*direction.value[1]
                        new_col_idx = col_idx + steps*direction.value[0]
                        if data[new_row_idx][new_col_idx] != char:
                            found_word = False
                            break

                    if found_word:
                        # Mark these all word locations as matched
                        for steps, char in enumerate(match_word):
                            new_row_idx = row_idx + steps*direction.value[1]
                            new_col_idx = col_idx + steps*direction.value[0]
                            matched_posn_counts[(new_col_idx, new_row_idx)] += 1
                        
                        matches += 1
    
    grid_rows = []
    for row_idx in range(grid_height):
        row_str = ""
        for col_idx in range(grid_width):
            row_str += data[row_idx][col_idx] if matched_posn_counts[(col_idx, row_idx)] > 0 else "."
        
        grid_rows.append(row_str)
    
    grid_vis = "\n".join(grid_rows)
    logger.info(f"\n{grid_vis}")

    return matches
    


%%time
sample_inputs = []
sample_inputs.append("""\
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX""")
sample_answers = [18]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")
### Day 4 Part 2

Quite different for part 2! Now we need to find any any X shape that contains "MAS" twice. E.g. a configuration like this:

```text
M.S
.A.
M.S
```

Observations:

- The centre character will always be `A`.
- The `M` and `S` characters each need to appear twice in the X.
- The `MAS` must be spelled diagonally. So if `M` is in one corner, then `S` will be in the opposite corner.

#### Solution Approach

- Set up a moveable X grid, by storing a list of vectors that represent the corners: NE, SE, SW, NW.
- Move the X through every possible location in the grid. We'll start by matching the `A` at the centre of the grid. For this reason, we should start at position `(1, 1)`, we will we will always stop one row / column before any given edge.
- Having matched an `A`, determine the coordinates of the four corners.
- Check that the four corners contain each remaining character (i.e. `M` or `S`) twice. If so, this location is a candidate.
- If we've found a candidate, determine the char in the top-right corner. Then determine if the char in the bottom-left corner is the _other_ end character in our word. If it is, then we've got a match. Why? Because if we've verified this diagonal, then by previously verifying the character counts, we know that the other diagonal is also satisfied.

And that's it! It's really quick too!

In [ ]:
def solve_part2(data):
    match_word = "MAS"
    center = match_word[1] # `A`
    ends = match_word[0] + match_word[-1] # `MS`

    grid_width = len(data[0])
    grid_height = len(data)

    matches = 0

    corner_vecs = [Vectors.NE.value,
                   Vectors.SE.value,
                   Vectors.SW.value,
                   Vectors.NW.value]

    # Traverse the grid, but always inset by 1
    for row_num in range(1, grid_height-1):
        for col_num in range(1, grid_width-1):
            current_centre = data[row_num][col_num]
            # Does middle of the X contain the A of "MAS"?
            if current_centre == center:
                corners = [(col_num+dx, row_num+dy) for dx, dy in corner_vecs]
                corner_chars = [data[y][x] for x, y in corners]

                # Check that each end char appears in the X exactly twice
                if all(corner_chars.count(char) == 2 for char in ends):
                    # if top right char is the opposite end char to bottom left char, we're matched
                    tr_char = data[corners[0][1]][corners[0][0]]
                    bl_char = data[corners[2][1]][corners[2][0]]
                    tr_match_idx = ends.index(tr_char)
                    if bl_char == ends[(len(ends)-tr_match_idx)-1]:
                        matches += 1

    return matches

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""\
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX""")
sample_answers = [9]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input.splitlines()), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

---
## Day 5: Print Queue

In [9]:
DAY = "5" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

#### See [Day 5](https://adventofcode.com/2024/day/5).

In [45]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

17:33:11.185:aoc2024 - DBG: input.txt already exists
17:33:11.187:aoc2024 - INF: Input data:
96|89
15|17
15|93
55|49
55|82
55|72
37|87
37|26
37|33
37|12
52|57
52|89
52|67
52|19
52|38
58|97
58|22
58|91
58|93
58|32
58|12
61|49
61|87
61|58
61|39
61|63
61|52
61|84
33|51
33|49
33|21
33|71
33|44
33|97
33|57
33|67
54|41
54|56
54|26
54|96
54|39
54|44
54|95
54|52
54|45
88|57
88|84
88|55
88|89
88|91
88|32
88|25
88|63
88|33
88|69
24|26
24|37
24|45
24|33
24|25
24|69
24|15
24|56
24|87
24|84
24|63
32|24
32|21
32|93
32|62
32|51
32|72
32|41
32|71
32|19
32|95
32|17
32|53
13|55
13|65
13|87
13|96
13|69
13|25
13|24
13|26
13|84
13|33
13|52
13|39
13|79
31|25
31|96
31|55
31|45
31|61
31|49
31|39
31|87
31|58
31|12
31|37
31|84
31|89
31|24
93|38
93|51
93|62
93|17
93|44
93|41
93|79
93|53
93|65
93|88
93|56
93|19
93|71
93|97
93|24
87|21
87|19
87|17
87|89
87|97
87|93
87|62
87|55
87|57
87|15
87|82
87|22
87|12
87|63
87|33
87|49
53|56
53|37
53|71
53|54
53|97
53|17
53|31
53|88
53|51
53|79
53|13
53|61
53|38
53|95
53|19
5

### Day 5 Part 1

Out input is in two blocks:

1. The page order rules - one rule per line. These are pairs of rules, e.g. `47|53`. This means that `if` an update contains *both* of these page numbers, then page `47` must be printed at some point *before* `53`.
1. The specified page numbers required in a particular update - one update set per line.

The sample input:

```text
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
```

**What is the sum of middle page numbers from only the updates are correctly ordered?**

#### Solution Approach

Rules for a given update:

- For any page number N, there must NOT be a rule `B|N` for any `B` that comes AFTER it.
- For any page number N, there must NOT be a rule `N|A` for any `A` that comes BEFORE it.

Notes:

- The rules appear to be a complete set, meaning that there wil be a rule for _every_ possible pair of pages. (?)
- A given update does necessarily include every page and therefore does not need to include every rule.

Rules that I probably don't need to check:

- For any page number N, there should be a rule `B|N` for every `B` that comes BEFORE it.
- For any page number N, there should be a rule `N|A` for every `A` that comes AFTER it.

Let's:

- Build a dictionary mapping 

In [ ]:
class PageOrdering():
    
    def __init__(self, data: str) -> None:
        
        required_after, required_before, updates = self._process_data(data)
        self._required_after = required_after
        self._required_before = required_before
        self._updates = updates
    
    @property
    def updates(self):
        return self._updates

    def _process_data(self, data: str):
        required_after = defaultdict(set)
        required_before = defaultdict(set)
        updates = [] 
        
        # split into blocks.
        rules_block, updates_block = data.split("\n\n") 
        
        # build two-way adjacency dictionary
        for rule in rules_block.splitlines():
            x, y = list(map(int, rule.split("|")))
            required_after[x].add(y)
            required_before[y].add(x)
        
        # Turn updates block into list of lists
        updates = [[int(x) for x in update_line.split(",")] for update_line in updates_block.splitlines()]
        
        return required_after, required_before, updates
    
    def is_correctly_ordered(self, update: list[int]):
        return self._test_comes_before(update)
    
    def _test_comes_before(self, update: list[int]):
        """
        - For any page number N, there should be a rule `B|N` for every `B` that comes BEFORE it.
        - For any page number N, there must NOT be a rule `B|N` for any `B` that comes AFTER it.
        - For any page number N, there should be a rule `N|A` for every `A` that comes AFTER it.
        - For any page number N, there must NOT be a rule `N|A` for any `A` that comes BEFORE it.
        """
        # E.g. 75,97,47,61,53
        for idx in range(len(update)):
            current_page = update[idx] # e.g. 75
            before = set(update[:idx])
            after = set(update[idx+1:]) # e.g. 97,47,61,53
            
            # set of required before pages
            required_before_page = self._required_before[current_page] # E.g. 97|75
            if intersect := after & required_before_page:
                return False
            
            required_after_page = self._required_after[current_page]
            if intersect := before & required_after_page:
                return False
            
        return True          
    
    def __repr__(self) -> str:
        return f"PageOrdering(rules_len={len(self._required_before)},updates_len={len(self._updates)})"

    def __str__(self) -> str:
        return "\n".join(",".join(map(str, update_row)) for update_row in self._updates)        
            

In [49]:
def solve_part1(data):
    page_ordering = PageOrdering(data)
    updates =page_ordering.updates
    
    sum_of_middle_pages = 0

    for update in updates:
        is_ordered = page_ordering.is_correctly_ordered(update)
        if is_ordered:
            middle = update[(len(update)//2)]
            logger.debug(f"{update} is in order; {middle=}")
            sum_of_middle_pages += middle
            
    return sum_of_middle_pages
    

In [50]:
%%time
sample_inputs = []
sample_inputs.append("""\
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47""")
sample_answers = [143]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

17:33:46.784:aoc2024 - DBG: [75, 47, 61, 53, 29] is in order; middle=61
17:33:46.785:aoc2024 - DBG: [97, 61, 53, 29, 13] is in order; middle=53
17:33:46.786:aoc2024 - DBG: [75, 29, 13] is in order; middle=29
17:33:46.787:aoc2024 - DBG: Rules fails because {97} is after 75
17:33:46.787:aoc2024 - DBG: Rules fails because {29} is after 13
17:33:46.788:aoc2024 - DBG: Rules fails because {75, 29, 47} is after 13
17:33:46.788:aoc2024 - INF: Test passed
17:33:46.788:aoc2024 - INF: All tests passed!
17:33:46.789:aoc2024 - DBG: Rules fails because {97, 38, 71, 72, 41, 44, 13, 17, 19, 51, 21, 22, 62, 95} is after 79
17:33:46.790:aoc2024 - DBG: [62, 97, 72, 71, 54, 95, 41, 51, 44, 13, 79, 31, 24, 37, 65, 56, 88, 61, 45, 39, 52] is in order; middle=79
17:33:46.791:aoc2024 - DBG: Rules fails because {97, 71, 72, 41, 44, 17, 19, 51, 21, 22, 54, 62, 95} is after 13
17:33:46.791:aoc2024 - DBG: [54, 95, 13, 79, 31, 24, 56, 88, 61, 47, 39, 84, 52, 96, 55] is in order; middle=88
17:33:46.792:aoc2024 - DB

CPU times: user 34.6 ms, sys: 39.3 ms, total: 73.9 ms
Wall time: 72.4 ms


### Day 5 Part 2

Overview...

In [ ]:
def solve_part2(data):
    pass

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""\
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47""")
sample_answers = [143]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")

---
## Day n: title

In [ ]:
DAY = "n" # replace with actual number (without leading digit)
day_link = f"#### See [Day {DAY}](https://adventofcode.com/{YEAR}/day/{DAY})."
display(Markdown(day_link))

In [ ]:
d_name = "d" + str(DAY).zfill(2) # e.g. d01
script_name = "aoc" + str(YEAR) + d_name # e.g. aoc2024d01
locations = dc.get_locations(d_name)
logger.setLevel(logging.DEBUG)
# td.setup_file_logging(logger, locations.output_dir)

# Retrieve input and store in local file
try:
    write_puzzle_input_file(YEAR, DAY, locations)
    with open(locations.input_file, mode="rt") as f:
        input_data = f.read().splitlines()

    logger.info("Input data:\n%s", dc.top_and_tail(input_data))
except ValueError as e:
    logger.error(e)

### Day n Part 1

Overview...

In [ ]:
def solve_part1(data):
    pass

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""abcdef""")
sample_answers = ["uvwxyz"]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part1(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")

logger.info("All tests passed!")

soln = solve_part1(input_data)
logger.info(f"Part 1 soln={soln}")

### Day n Part 2

Overview...

In [ ]:
def solve_part2(data):
    pass

In [ ]:
%%time
sample_inputs = []
sample_inputs.append("""abcdef""")
sample_answers = ["uvwxyz"]

for curr_input, curr_ans in zip(sample_inputs, sample_answers):
    validate(solve_part2(curr_input), curr_ans) # test with sample data
    logger.info("Test passed")    

logger.info("Tests passed!")

soln = solve_part2(input_data)
logger.info(f"Part 2 soln={soln}")